In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#Scraping Wikipedia for the raw data
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(website_url, 'lxml')
table = soup.find('table', {'class':'wikitable sortable'})

In [3]:
#Extracting the Table Headers
headers = table.findAll('th')
headers_result = []
for i in range(len(headers)):
    s = str(headers[i])
    start = s.find('<th>') + len('<th>')
    end = s.find('</th>')
    headers_result.append(s[start:end-1])
    

#Extracting the data from the table
data = table.findAll('td')
data_result = []
for i in range(len(data)):
    s = str(data[i])
    start = s.find('<td>') + len('<td>')
    end = s.find('</td>')
    data_result.append(s[start:end-1])

In [16]:
#Creating the DataFrame and populating
df = pd.DataFrame(columns = headers_result) 
num_cols = len(headers_result)
num_rows = int(len(data_result)/num_cols)

i = 0 

for row_num in range(num_rows):
    pc = data_result[i]
    borough = data_result[i+1]
    neigh = data_result[i+2]
    
    #removing not assigned borough
    if borough=='Not assigned':
        i+=3
        next
    else:
        df.loc[row_num] = [pc, borough, neigh]
        i+=3

df.reset_index(drop=True, inplace=True)
#df.head()

In [17]:
df['Neighborhood'].replace('/', ',', regex=True, inplace=True)
#df.head()

In [6]:
for i in range(len(df)):
    if df['Neighborhood'][i] == 'Not assigned':
        df['Neighborhood'][i] = df['Borough'][i]

In [7]:
df.shape

(103, 3)

# Start of Part 2 

Reading CSV file for coordinates

In [8]:
geodata_df = pd.read_csv('http://cocl.us/Geospatial_data')
geodata_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Renaming the header to accomodate capitalization in order to perform merge on postal code as key

In [9]:
geodata_df.rename(columns={'Postal Code': 'Postal code'},inplace = True)
df_merged = pd.merge(df, geodata_df, on = 'Postal code')

In [12]:
df_merged

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509
